<a href="https://colab.research.google.com/github/mikendonga/Medical-Chatbot/blob/main/NLP_medical_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from difflib import get_close_matches
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
#load dataset
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("choongqianzheng/disease-and-symptoms-dataset")

print("Path to dataset files:", path)
print(os.listdir(path))

file_path =os.path.join(path, 'DiseaseAndSymptoms.csv')
df = pd.read_csv(file_path)

Using Colab cache for faster access to the 'disease-and-symptoms-dataset' dataset.
Path to dataset files: /kaggle/input/disease-and-symptoms-dataset
['DiseaseAndSymptoms.csv', 'Disease precaution.csv']


In [ ]:
df = pd.read_csv(file_path)
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#data preprocessing
class SymptomDataPreprocessor:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        # Keep important negation words
        self.stop_words = self.stop_words - {'no', 'not', 'never', 'without'}

    def preprocess_dataset(self, df):
        """

        """
        # Create a copy to avoid modifying original data
        processed_df = df.copy()

        # 1. Handle missing values
        processed_df = self._handle_missing_values(processed_df)

        # 2. Clean and preprocess text columns
        processed_df = self._preprocess_text_columns(processed_df)

        # 3. Feature engineering
        processed_df = self._create_features(processed_df)

        return processed_df

    def _handle_missing_values(self, df):
        """Handle missing values in the dataset"""
        # Fill missing values with empty strings for text columns
        text_columns = df.select_dtypes(include=['object']).columns
        df[text_columns] = df[text_columns].fillna('')

        # For numeric columns, you might use different strategies
        numeric_columns = df.select_dtypes(include=[np.number]).columns
        df[numeric_columns] = df[numeric_columns].fillna(0)

        return df

    def _preprocess_text_columns(self, df):
        """Preprocess all text columns in the dataset"""

        # Assuming the dataset has columns like 'symptoms', 'disease', etc.
        # Adjust based on actual column names

        text_columns = ['symptoms', 'description']  # Adjust based on your dataset

        for col in text_columns:
            if col in df.columns:
                print(f"Preprocessing column: {col}")
                df[col] = df[col].apply(self._advanced_text_cleaning)

        return df

    def _advanced_text_cleaning(self, text):
        """Advanced text cleaning with medical context"""
        if pd.isna(text) or text == '':
            return ""

        # Convert to lowercase
        text = text.lower()

        # Remove special characters but keep important medical symbols
        text = re.sub(r'[^\w\s\-+/]', '', text)

        # Handle common medical abbreviations
        text = self._standardize_medical_terms(text)

        # Tokenize
        tokens = word_tokenize(text)

        # Remove stopwords and short tokens
        tokens = [token for token in tokens
                 if token not in self.stop_words and len(token) > 2]

        # Lemmatize
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens]

        return ' '.join(tokens)

    def _standardize_medical_terms(self, text):
        """Standardize common medical terms and abbreviations"""
        # Common medical term mappings
        medical_mappings = {
            r'\bfeverish\b': 'fever',
            r'\bheadache\b': 'headache',
            r'\bnauseous\b': 'nausea',
            r'\bvomiting\b': 'vomit',
            r'\bdiarrhea\b': 'diarrhoea',  # Standardize spelling
            r'\bfatigue\b': 'tired',
            r'\bfatigued\b': 'tired',
            r'\bpainful\b': 'pain',
        }

        for pattern, replacement in medical_mappings.items():
            text = re.sub(pattern, replacement, text)

        return text

    def _create_features(self, df):
        """Create additional features for ML models"""

        # Symptom count (if symptoms are in a list or can be counted)
        if 'symptoms' in df.columns:
            df['symptom_count'] = df['symptoms'].apply(
                lambda x: len(str(x).split(',')) if pd.notna(x) else 0
            )

        # Text length features
        text_columns = ['symptoms', 'description']
        for col in text_columns:
            if col in df.columns:
                df[f'{col}_length'] = df[col].apply(
                    lambda x: len(str(x).split()) if pd.notna(x) else 0
                )

        return df


In [ ]:
#split into features and labels
x = df.drop("Disease",axis=1)
y = df["Disease"]

# One-hot encode the symptom features
x = pd.get_dummies(x, prefix_sep='_')

#label encoding
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

#train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# Model
model = RandomForestClassifier(n_estimators=300, random_state=42)
model.fit(x_train, y_train)

RandomForestClassifier(n_estimators=300, random_state=42)

In [ ]:
import csv

#Dictionaries
severityDictionary = {}
description_list = {}
precautionDictionary = {}
symptoms_dict = {symptom: idx for idx, symptom in enumerate(x)}

# Define the base directory for your master data files
# You need to ensure 'symptom_Description.csv', 'symptom_severity.csv', and 'symptom_precaution.csv' are in this directory.
# For example, if you uploaded them to a folder named 'MasterData' in your Colab files,
# set MASTER_DATA_DIR = 'MasterData'
# If they are in the root of your Colab environment, set MASTER_DATA_DIR = '.'
MASTER_DATA_DIR = '/content/MasterData' # <<<--- ADJUST THIS PATH IF YOUR FILES ARE ELSEWHERE

def getDescription():
    # Use os.path.join for robust path construction
    file_path = os.path.join(MASTER_DATA_DIR, 'symptom_Description.csv')
    with open(file_path) as csv_file:
        for row in csv.reader(csv_file):
            description_list[row[0]] = row[1]

def getSeverityDict():
    # Use os.path.join for robust path construction
    file_path = os.path.join(MASTER_DATA_DIR, 'Symptom_severity.csv') # Corrected filename here
    with open(file_path) as csv_file:
        for row in csv.reader(csv_file):
            try:
                severityDictionary[row[0]] = int(row[1])
            except:
                pass

def getprecautionDict():
    # Use os.path.join for robust path construction
    file_path = os.path.join(MASTER_DATA_DIR, 'symptom_precaution.csv')
    with open(file_path) as csv_file:
        for row in csv.reader(csv_file):
            precautionDictionary[row[0]] = [row[1], row[2], row[3], row[4]]

In [ ]:
#user input symptoms
def extract_symptoms(user_input, all_symptoms):
    extracted = []
    text = user_input.lower().replace("-", " ")

    # 1. Synonym replacement
    for phrase, mapped in symptom_synonyms.items():
        if phrase in text:
            extracted.append(mapped)

    # 2. Exact match
    for symptom in all_symptoms:
        if symptom.replace("_", " ") in text:
            extracted.append(symptom)

    # 3. Fuzzy match (typo handling)
    words = re.findall(r"\w+", text)
    for word in words:
        close = get_close_matches(word, [s.replace("_", " ") for s in all_symptoms], n=1, cutoff=0.8)
        if close:
            for sym in all_symptoms:
                if sym.replace("_", " ") == close[0]:
                    extracted.append(sym)

    return list(set(extracted))


In [ ]:
# ------------------ Prediction ------------------
def predict_disease(raw_symptoms_list):
    input_vector = np.zeros(len(symptoms_dict))
    # Iterate through all possible one-hot encoded symptom features in symptoms_dict
    for oh_symptom_key, idx in symptoms_dict.items():
        # Check if any of the raw symptoms from the user input match part of the one-hot encoded key
        for raw_symptom in raw_symptoms_list:
            # Example: if raw_symptom is 'chest_pain' and oh_symptom_key is 'Symptom_1_chest_pain'
            # We check if '_chest_pain' is in 'Symptom_1_chest_pain'
            if f'_{raw_symptom}' in oh_symptom_key:
                input_vector[idx] = 1
                break # Found a match for this oh_symptom_key, move to the next one

    pred_proba = model.predict_proba([input_vector])[0]
    pred_class = np.argmax(pred_proba)
    disease = le.inverse_transform([pred_class])[0]
    confidence = round(pred_proba[pred_class] * 100, 2)
    return disease, confidence, pred_proba


In [ ]:
# Helper data for symptom extraction
# Define cols (all unique raw symptom names)
all_raw_symptoms = []
# Assuming symptoms are in columns Symptom_1 to Symptom_17
for i in range(1, 18):
    col_name = f'Symptom_{i}'
    if col_name in df.columns: # Check if column exists in df
        # Extend with non-NaN unique values
        all_raw_symptoms.extend(df[col_name].dropna().unique())
cols = sorted(list(set(all_raw_symptoms)))

# Define symptom_synonyms
symptom_synonyms = {
    "chest pain": "chest_pain",
    "fever": "high_fever",
    "stomach ache": "stomach_pain",
    "vomiting": "vomiting",
    "rash": "skin_rash",
    "cold": "cold_hands_and_feets", # Example, add more as needed
    "pain in joints": "joint_pain", # Example
    "dizziness": "dizziness",
    "fatigue": "fatigue",
    "cough": "cough",
    "sneezing": "continuous_sneezing",
    "diarrhea": "diarrhoea",
    "sore throat": "sore_throat",
    "runny nose": "runny_nose"
}

#  Chatbot
def chatbot():
    getSeverityDict()
    getDescription()
    getprecautionDict()

    print("🤖 Welcome to HealthCare ChatBot")
    print("Hello! Please answer a few questions so I can understand your condition better.")

    # Basic info
    name = input("👉 What is your name? : ")
    age = input("👉 Please enter your age: ")
    gender = input("👉 What is your gender? (M/F/Other): ")

    # Initial symptoms
    symptoms_input = input("👉 Describe your symptoms in a sentence (e.g., 'I have fever and stomach pain'): ")
    symptoms_list = extract_symptoms(symptoms_input, cols)

    if not symptoms_list:
        print("❌ Sorry, I could not detect valid symptoms. Please try again with more details.")
        return

    print(f"✅ Detected symptoms: {', '.join(symptoms_list)}")

    num_days = int(input("👉 For how many days have you had these symptoms? : "))
    severity_scale = int(input("👉 On a scale of 1–10, how severe do you feel your condition is? : "))
    pre_exist = input("👉 Do you have any pre-existing conditions (e.g., diabetes, hypertension)? : ")
    lifestyle = input("👉 Do you smoke, drink alcohol, or have irregular sleep? : ")
    family = input("👉 Any family history of similar illness? : ")

    # ---------------- Initial Prediction ----------------
    disease, confidence, proba = predict_disease(symptoms_list)

    # ---------------- Guided Questions (Disease-specific) ----------------
    print("\n🤔 Let me ask you some more questions related to", disease)
    # The 'training' DataFrame is not defined here. Assuming it refers to the original DataFrame 'df' or a processed version.
    # For this example, I'll use the original df to get disease-specific symptoms if 'training' is not available.
    # If 'training' is intended, it needs to be defined in the global scope.
    # As a placeholder, let's assume 'df' has 'Disease' and symptom columns for getting disease-specific symptoms.
    # This part might need further adjustment based on how 'training' was intended to be used.

    # Reconstructing disease_symptoms logic based on typical disease prediction flows
    # This assumes we want to find symptoms associated with the predicted 'disease' from the original df
    # This section might be simplified or require a 'training' DataFrame that holds binary symptom features per disease
    disease_symptoms_from_df = []
    if 'Disease' in df.columns:
        # Find rows corresponding to the predicted disease
        disease_rows = df[df['Disease'] == disease]
        if not disease_rows.empty:
            # Get all symptom columns (Symptom_1 to Symptom_17)
            symptom_cols = [f'Symptom_{i}' for i in range(1, 18) if f'Symptom_{i}' in df.columns]
            # Collect all unique symptoms for this disease
            for col in symptom_cols:
                disease_symptoms_from_df.extend(disease_rows[col].dropna().unique())
            disease_symptoms_from_df = list(set(disease_symptoms_from_df))

    # Use the collected symptoms for guided questions
    asked = 0
    for sym in disease_symptoms_from_df:
        if sym not in symptoms_list and asked < 8:
            ans = input(f"👉 Do you also have {sym.replace('_',' ')}? (yes/no): ").strip().lower()
            if ans == "yes":
                symptoms_list.append(sym)
            asked += 1


In [ ]:

#  Run chatbot
if __name__ == "__main__":
    chatbot()

🤖 Welcome to HealthCare ChatBot
Hello! Please answer a few questions so I can understand your condition better.


In [ ]:
import os

master_data_path = '/content/MasterData'
if os.path.exists(master_data_path):
    print(f"Contents of {master_data_path}:")
    for item in os.listdir(master_data_path):
        print(item)
else:
    print(f"Directory '{master_data_path}' does not exist.")

Contents of /content/MasterData:
symptom_Description.csv
Symptom_severity.csv
symptom_precaution.csv
